In [1]:
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 280

import matplotlib.pyplot as plt


import nltk
from nltk.probability import FreqDist
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
sw = stopwords.words('english')



In [2]:
import sys
sys.path.append( '../../src' )
from pandas_functions import *

In [3]:
dataFolder_path = '../../data/'

In [4]:
data_df = pd.read_csv(dataFolder_path+'judge_1377884607_tweet_product_company.csv')

In [5]:
# def get_wordnet_pos(treebank_tag):
#     '''
#     Translate nltk POS to wordnet tags
#     '''
#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN


def doc_preparer(doc, stop_words=sw):
    '''

    :param doc: a document from the satire corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    #Stemming seems to work better. Lemming can't identify plurals of products
    
    
#     lemmed_keywords = ['apple',
#                 'ipad', 'ipads',
#                 'iphone', 'iphones',
#                 'itunes',
#                 'google', 'googled',
#                 'android', 'droid', 'androids', 'droids',
#                 'circle', 'circles'
#                 'app', 'apps']

    stemmed_keywords = ['appl',
                        'ipad',
                        'iphon',
                        'itun',
                        'googl',
                        'android',
                        'droid',
                        'circl',
                        'app']
    
    
    #Updating SW words to include some Twitter specific stopwords and sxsw words
    twitter_specific_sw = ['rt','@mention']

    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
#     doc = pos_tag(doc)
#     doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
#     lemmatizer = WordNetLemmatizer()
#     doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
#     doc = [word for word in doc if word in lemmed_keywords]
    
    
    
    p_stemmer = nltk.stem.PorterStemmer()
    doc = [word for word in doc if p_stemmer.stem(word) in stemmed_keywords]
    return doc

In [6]:
# data_df['keywords_mentioned'] = data_df['tweet_text'].map(lambda x:doc_preparer(x))

In [16]:
# drop_index = data_df[
#     (data_df.keywords_mentioned.str.len()==0)  &
#        (data_df.emotion_in_tweet_is_directed_at.notnull())].index

# data_df.drop(drop_index,inplace=True)

In [7]:
dataFrame_info(data_df)

Datframe has 8721 rows and 3 columns


Info Table:                                        Zeroes          Nulls  \
Details:                                            Count Fraction Count   
Columns:                                                                   
tweet_text                                              0   0.00 %     1   
emotion_in_tweet_is_directed_at                         0   0.00 %  5552   
is_there_an_emotion_directed_at_a_brand_or_product      0   0.00 %     0   

Info Table:                                                 Uniques           \
Details:                                           Fraction   Count Fraction   
Columns:                                                                       
tweet_text                                           0.01 %    8694  99.69 %   
emotion_in_tweet_is_directed_at                     63.66 %      10   0.11 %   
is_there_an_emotion_directed_at_a_brand_or_product   0.00 %       4   0.05 %   

Info Table:                                        Missing/Unknown           \
Details:                                                     Count Fraction   
Columns:                                                                      
tweet_text                                                       0   0.00 %   
emotion_in_tweet_is_directed_at                                  0   0.00 %   
is_there_an_emotion_directed_at_a_brand_or_product               0   0.00 %   

Info Table:                                        Mean Median  
Details:                                                        
Columns:                                                        
tweet_text                                          0.0    0.0  
emotion_in_tweet_is_directed_at                     0.0    0.0  
is_there_an_emotion_directed_at_a_brand_or_product  0.0    0.0

Looking at the 1 null in tweet text

In [8]:
data_df[data_df.tweet_text.isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
6,NaN,NaN,No emotion toward brand or product


In [9]:
data_df.dropna(subset=['tweet_text'],inplace=True)

In [10]:
data_df.shape

(8720, 3)

In [11]:
dataFrame_info(data_df)

Datframe has 8720 rows and 3 columns


Info Table:                                        Zeroes          Nulls  \
Details:                                            Count Fraction Count   
Columns:                                                                   
tweet_text                                              0   0.00 %     0   
emotion_in_tweet_is_directed_at                         0   0.00 %  5551   
is_there_an_emotion_directed_at_a_brand_or_product      0   0.00 %     0   

Info Table:                                                 Uniques           \
Details:                                           Fraction   Count Fraction   
Columns:                                                                       
tweet_text                                           0.00 %    8693  99.69 %   
emotion_in_tweet_is_directed_at                     63.66 %      10   0.11 %   
is_there_an_emotion_directed_at_a_brand_or_product   0.00 %       4   0.05 %   

Info Table:                                        Missing/Unknown           \
Details:                                                     Count Fraction   
Columns:                                                                      
tweet_text                                                       0   0.00 %   
emotion_in_tweet_is_directed_at                                  0   0.00 %   
is_there_an_emotion_directed_at_a_brand_or_product               0   0.00 %   

Info Table:                                        Mean Median  
Details:                                                        
Columns:                                                        
tweet_text                                          0.0    0.0  
emotion_in_tweet_is_directed_at                     0.0    0.0  
is_there_an_emotion_directed_at_a_brand_or_product  0.0    0.0

Lets look at the emotion quotient column

In [12]:
data_df.is_there_an_emotion_directed_at_a_brand_or_product.value_counts(normalize=True)

No emotion toward brand or product    0.591170
Positive emotion                      0.329014
Negative emotion                      0.062500
I can't tell                          0.017317
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: float64

Major class imbalance. Should consider dropping "I can't tell". 

In [13]:
data_df[data_df.is_there_an_emotion_directed_at_a_brand_or_product == "I can't tell" ]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
88,Thanks to @mention for publishing the news of @mention new medical Apps at the #sxswi conf. blog {link} #sxsw #sxswh,NaN,I can't tell
100,���@mention &quot;Apple has opened a pop-up store in Austin so the nerds in town for #SXSW can get their new iPads. {link} #wow,NaN,I can't tell
228,"Just what America needs. RT @mention Google to Launch Major New Social Network Called Circles, Possibly Today {link} #sxsw",NaN,I can't tell
330,The queue at the Apple Store in Austin is FOUR blocks long. Crazy stuff! #sxsw,NaN,I can't tell
357,Hope it's better than wave RT @mention Buzz is: Google's previewing a social networking platform at #SXSW: {link},NaN,I can't tell
...,...,...,...
8650,It's funny watching a room full of people hold their iPad in the air to take a photo. Like a room full of tablets staring you down. #SXSW,NaN,I can't tell
8662,"@mention yeah, we have @mention , Google has nothing on us :) #SXSW",NaN,I can't tell
8667,"@mention Yes, the Google presentation was not exactly what I was expecting. #sxsw",NaN,I can't tell
8687,&quot;Do you know what Apple is really good at? Making you feel bad about your Xmas present!&quot; - Seth Meyers on iPad2 #sxsw #doyoureallyneedthat?,NaN,I can't tell


In [14]:
data_df.shape

(8720, 3)

In [ ]:
data_df.is_there_an_emotion_directed_at_a_brand_or_product.unique()

In [ ]:
data_df['tweet_text'].dtype

In [ ]:
kw_mentioned = set()
for each in data_df.keywords_mentioned:
    kw_mentioned.update(set(each))

In [ ]:
data_df.keywords_mentioned

In [ ]:
kw_mentioned

In [ ]:
t = "Thanks to @mention for publishing the news of @mention new medical Apps at the #sxswi conf. blog {link} #sxsw #sxswh	"

In [ ]:
regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
doc = regex_token.tokenize(t)
# doc = [word.lower() for word in doc]
# doc = [word for word in doc if word not in sw]

doc